In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/fraud-detection/synth_composite.csv


## Import Libraries

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

sns.set_theme()
sns.set_style("darkgrid")

## Load dataset

In [3]:
df = pd.read_csv('../input/fraud-detection/synth_composite.csv')

### Show some stats

In [4]:
df.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,211,CASH_OUT,184185.755277,160_2696646_0,203108.476051,18922.720773,160_901564_0,1.019994e+06,1.204127e+06,0,0
1,394,CASH_OUT,237093.196009,160_5585569_0,382707.464238,145614.268230,160_1586945_0,5.870893e+05,8.243203e+05,0,0
2,306,CASH_IN,163600.518794,160_4221372_0,3093.431169,0.000000,160_1701821_0,6.534538e+05,4.901219e+05,0,0
3,235,PAYMENT,7625.866018,160_3064357_0,74630.853812,67004.987793,160_1304279_0,1.666019e+01,1.362096e+01,0,0
4,235,PAYMENT,7866.992466,160_5544127_0,74632.326454,66765.333988,160_2370260_0,5.390545e+00,9.820910e+00,0,0
5,493,CASH_OUT,281403.795998,160_6037426_0,42235.923347,0.000000,160_2092767_0,5.880235e+05,8.693219e+05,0,0
6,18,PAYMENT,4355.986182,160_400626_0,56160.034248,51804.048066,160_175474_0,8.496485e+00,1.078249e+01,0,0
7,550,CASH_IN,102079.164541,160_6146482_0,15781.717821,0.000000,160_354507_0,4.064748e+06,3.962757e+06,0,0
8,284,CASH_OUT,144074.648278,160_3889478_0,197270.864524,53196.216246,160_1089660_0,1.097389e+06,1.241346e+06,0,0
9,132,CASH_OUT,142367.900524,160_1163767_0,0.185064,0.000000,160_29365_0,2.424877e+06,2.567106e+06,0,0


In [5]:
df.shape

(6362620, 11)

In [6]:
df.sample(frac=0.1).shape

(636262, 11)

In [7]:
df.shape

(6362620, 11)